In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# pd.set_option('display.max_colwidth', None)

2024-08-06 15:37:45.928735: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-06 15:37:45.929089: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 15:37:45.931156: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 15:37:45.958131: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-06 15:37:46.490340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [ ]:
df = pd.read_json("~/Documents/Documents/Projects/LLMs/review rating/Beauty_5_50000.json", lines=True)

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df['reviewText'][0]

In [ ]:
df['reviewText'][1]

In [ ]:
df['reviewText'][2]

In [ ]:
df['reviewText'][3]

In [ ]:
df['reviewText'][4]

# Punctuation Removal

In [ ]:
def remove_punctuation(text):
    punctuationFree = "".join([i for i in text if i not in string.punctuation])
    return punctuationFree

df["puncFree"] = df['reviewText'].apply(lambda x:remove_punctuation(x))

In [ ]:
df['reviewText'][0]

In [ ]:
df['puncFree'][0]

# Lowercasing

In [ ]:
df["lowercased"] = df["puncFree"].apply(lambda x: x.lower())

In [ ]:
df['lowercased'][0]

# Tokenization

In [ ]:
def tokenization(text):
    tokens = re.split("\W+",text)
    return tokens

df["tokenized"] = df["lowercased"].apply(lambda x: tokenization(x))

In [ ]:
np.array(df['tokenized'][0])

# stop-word Removal

In [ ]:
stopWords = nltk.corpus.stopwords.words('english')
stopWords[0:10]

In [ ]:
def remove_stopwords(text):
    output = [i for i in text if i not in stopWords]
    return output

df["no_stop_words"] = df["tokenized"].apply(lambda x: remove_stopwords(x))

In [ ]:
np.array(df['no_stop_words'][0])

In [ ]:
np.array(df['no_stop_words'][1])

# Lemmatization

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

df['lemmatized'] = df['no_stop_words'].apply(lambda x:lemmatizer(x))

In [ ]:
np.array(df['lemmatized'][0])

In [ ]:
np.array(df['lemmatized'][1])

In [ ]:
df['lemmatized'] = [item for item in df['lemmatized'] if not isinstance(item, int)]

In [ ]:
df_new = df.copy()
df_new = df_new.drop(columns=['asin','helpful','reviewText','tokenized','no_stop_words', 'reviewTime',
                             'reviewerID', 'reviewerName', 'summary', 'unixReviewTime', 'puncFree',
                             'lowercased'])

In [ ]:
df_new.head(2)

In [ ]:
# for item in df['lemmatized'].iterrows():
#     if not isinstance(item, int)

# Data Spliting

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_new['lemmatized'], df_new['overall'],
                                                   test_size=0.1, random_state=42)

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
x_test

In [ ]:
y_test

# Update the internal Vocabulary

In [ ]:
#Tokenize the sentences
tokenizer_x_train = Tokenizer()
#Updates internal vocabulary based on a list of texts.
tokenizer_x_train.fit_on_texts(x_train)

tokenizer_y_train = Tokenizer()
tokenizer_y_train.word_index = tokenizer_x_train.word_index

first2pairs = {k: tokenizer_x_train.word_index[k] for k in list(tokenizer_x_train.word_index)[:10]}
first2pairs

In [ ]:
tf.sysconfig.get_build_info()

In [ ]:
len(list(tokenizer_x_train.word_index))

# Sequencing and Padding

In [ ]:
#Transforms each text in texts to a sequence of integers.
x_train_seq = tokenizer_x_train.texts_to_sequences(x_train)
y_train_seq = tokenizer_y_train.texts_to_sequences(y_train)

#Pads sequences to the same length.
x_train_pad = pad_sequences(x_train_seq, padding='post')
y_train_pad = pad_sequences(y_train_seq, padding='post', maxlen=197)

# Create Attention Mask

In [ ]:
# Function to create attention masks
def create_attention_mask(sequences):
    attention_masks = tf.math.not_equal(sequences, 0)
    return tf.cast(attention_masks, dtype=tf.int32)

# Create attention masks for both x_train_pad and y_train_pad
x_train_attention_mask = create_attention_mask(x_train_pad).numpy()
y_train_attention_mask = create_attention_mask(y_train_pad).numpy()

# Print the attention masks for verification
print("x_train_attention_mask:")
print(x_train_attention_mask)

print("\ny_train_attention_mask:")
print(y_train_attention_mask[0])